In [1]:
import os
import json
import pandas as pd

In [2]:
def truncate_review_text(line):
    # Find the index of "review/text" in the line
    line = "{" + line[line.find('"beer/beerId"') :]
    line = line[: line.find('"beer/name"')] + line[line.find('"beer/beerId"') + 1 :]
    line = line[: line.find('"review/text"') - 2] + "}"
    return line

In [3]:
json_file_path = os.path.join("..", "..", "data", "ratebeer", "ratebeer.json")
prime = []
count = 0
count_1 = 0
for line in open(json_file_path, "r"):
    line = line.replace("'", '"')
    line = truncate_review_text(line)
    if line == "{}\n":
        continue
    try:
        val = json.loads(line)
        prime.append(
            [
                val["review/profileName"],
                val["beer/beerId"],
                val["review/overall"],
                val["review/time"],
            ]
        )
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
        print(f"Line: {line}")
        continue

JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Line: }


In [4]:
ratings = pd.DataFrame(prime, columns=["userId", "itemId", "rating", "timestamp"])

In [5]:
ratings["rating"] = ratings["rating"].apply(
    lambda rating_str: float(rating_str.split("/")[0])
    / float(rating_str.split("/")[1])
    * 5
)
ratings.timestamp = ratings.timestamp.astype(int)

In [ ]:
ratings.to_parquet(
    os.path.join("..", "..", "data", "ratebeer", "ratings.parquet"), index=False
)